In [1]:
import pandas as pd
import os 

In [4]:
ruta_archivos = "D:/PROYECTO FINAL/Google Maps/metadata-sitios/"     # Ruta de los json de Sitios
archivos_json = [[os.path.join(ruta_archivos, archivo), archivo] for archivo in os.listdir(ruta_archivos) if archivo.endswith('.json')]

In [5]:
archivos_json

[['D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json', '1.json'],
 ['D:/PROYECTO FINAL/Google Maps/metadata-sitios/2.json', '2.json']]

In [24]:
df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")

In [2]:
df_sitios= pd.read_json('D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json', lines=True)

In [3]:
df_sitios= df_sitios.dropna(subset=['address']) # Elimina filas con address nulas
df_sitios= df_sitios.dropna(subset=['category']) # Elimina filas con category nulas

In [4]:
df_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 262678 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              262678 non-null  object 
 1   address           262678 non-null  object 
 2   gmap_id           262678 non-null  object 
 3   description       13063 non-null   object 
 4   latitude          262678 non-null  float64
 5   longitude         262678 non-null  float64
 6   category          262678 non-null  object 
 7   avg_rating        262678 non-null  float64
 8   num_of_reviews    262678 non-null  int64  
 9   price             13319 non-null   object 
 10  hours             183150 non-null  object 
 11  MISC              192649 non-null  object 
 12  state             186168 non-null  object 
 13  relative_results  229847 non-null  object 
 14  url               262678 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 32.1+ MB


In [5]:
df_sitios= df_sitios.explode('category')

In [12]:
df_sitios= df_sitios[df_sitios['category'] == "Restaurant"]

In [13]:
# Se guardan las columnas MISC y el gmap_id en un df
df_misc= df_sitios[['gmap_id','MISC']]
df_misc= df_misc.dropna(subset=['MISC']) # Elimina filas con MISC nulo

In [18]:
# Funcion para cargar los json a una serie de pandas
def expandir_diccionario(diccionario):
    return pd.Series(diccionario)

# Se concatenas las series al df original
df_expandido = pd.concat([df_misc, df_misc['MISC'].apply(expandir_diccionario)], axis=1)
df_expandido.drop('MISC', axis=1, inplace=True)     # Se elimina la columna de MISC
df_misc= df_expandido

In [20]:
df_Service_options= df_misc[['gmap_id','Service options']]      # Se crea un df solo las columnas gmap_id y Service Options
df_Service_options= df_Service_options.rename(columns={'Service options': 'service_option'}) # cambiar nombre de la columna
df_Service_options= df_Service_options.dropna(subset=['service_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Service_options= df_Service_options.explode('service_option')

In [29]:
df_Service_options

,gmap_id,service_option
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Outdoor seating
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Curbside pickup
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No-contact delivery
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Delivery
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Takeout
...,...,...
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,No-contact delivery
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Delivery
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Takeout
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Dine-in


In [22]:
df_Planning= df_misc[['gmap_id','Planning']]      # Se crea un df solo las columnas gmap_id y Planning
df_Planning= df_Planning.rename(columns={'Planning': 'planning_option'}) # cambiar nombre de la columna
df_Planning= df_Planning.dropna(subset=['planning_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Planning= df_Planning.explode('planning_option')

In [30]:
df_Planning

,gmap_id,planning_option
307,0x86275dc7cfc138d5:0x4c35fe08b16e7a2b,Accepts reservations
917,0x89c587762e7e69dd:0x30a60dce48b073c3,Accepts reservations
1165,0x88c2fea614c52c8b:0x50e166caa05e8c0f,Accepts reservations
1665,0x89e80e710fddbabb:0xdaa5f6da10c030b9,Accepts reservations
2147,0x89c25a275e5088dd:0xb6ecb4eaa82d1180,Quick visit
...,...,...
274086,0x89c2c70157450c41:0xd604da267031b8bc,Dinner reservations recommended
274086,0x89c2c70157450c41:0xd604da267031b8bc,Accepts reservations
274311,0x80c8daee5baf656d:0xd86ef2a8bfdb748b,Accepts reservations
274785,0x88f45b888accca4f:0xf5341d15da635764,Accepts reservations


In [ ]:
# En pruebas se detecto que esta fila estaba dando problemas con la funcion para extraer, asi que se procede a eliminarla
df_sitios= df_sitios[df_sitios['address'] != "〒10028 New York, Lexington Ave, (New) Ichie Japanese Restaurant"]

In [25]:
def Ext_Ciudad_Estado(dir):
    ciudad= "SIN DATO"
    estado= "SIN DATO"     
    if len(str(dir)) > 10:  
        lista= str(dir).split(',')
        if len(lista) > 2:
            codigo= lista[-1][1:3]
            df_filtro= df_estados_USA[df_estados_USA['nombre_corto'].str.contains(codigo)]        
            if not df_filtro.empty:            
                ciudad = lista[-2].strip()
                estado= df_filtro.nombre_largo.values[0].strip()
    return ciudad, estado

In [26]:
df_sitios[['ciudad','estado']] = df_sitios.apply(lambda x: Ext_Ciudad_Estado(x['address']), axis=1, result_type='expand')

C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_8188\4292867449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sitios[['ciudad','estado']] = df_sitios.apply(lambda x: Ext_Ciudad_Estado(x['address']), axis=1, result_type='expand')
C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_8188\4292867449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sitios[['ciudad','estado']] = df_sitios.apply(lambda x: Ext_Ciudad_Estado(x['address']), axis=1, result_type='expand')


In [27]:
#Borrar Columnas
df_sitios= df_sitios.drop(['relative_results','address', 'num_of_reviews', 'description', 'url','category', 'MISC', 'hours'], axis=1) 
# Ordena el orden de las columnas
df_sitios= df_sitios[['gmap_id','name', 'ciudad', 'estado', 'latitude', 'longitude', 'avg_rating', 'price', 'state']]

In [28]:
df_sitios

,gmap_id,name,ciudad,estado,latitude,longitude,avg_rating,price,state
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,California,33.916402,-118.010855,4.5,None,Open ⋅ Closes 9:30PM
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Vivi Bubble Tea,Philadelphia,Pennsylvania,39.940293,-75.150923,4.0,None,None
96,0x7c00456eecad3111:0x8217f9600c51f33,Hale Pops,Laie,Hawái,21.637796,-157.920714,4.4,None,Closed ⋅ Opens 11AM
114,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,Cape Coral,Florida,26.641377,-81.940545,5.0,$$,None
123,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,Texas,30.273985,-97.719563,4.5,None,Closed ⋅ Opens 5PM
...,...,...,...,...,...,...,...,...,...
274875,0x879984bc08a64061:0x7ab948f1bbff7c0e,Quaker Steak & Lube,Grand Island,Nebraska,40.830827,-98.380752,2.9,None,None
274895,0x8838629ba011ece9:0x9b75b19079ad45e7,Subway,Whitehall,Ohio,39.954151,-82.900727,3.7,$,Open ⋅ Closes 8PM
274896,0x884fa3f049375197:0xf7ea720f164ef342,ON THE RIVER SPORTS BAR AND GRILL,Gilbert,West Virginia,37.601291,-81.860280,4.4,None,Permanently closed
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Palermo’s Market,Rochester,New York,43.157746,-77.605956,4.9,None,Open ⋅ Closes 7PM
